# 1.创建Tensor

In [2]:
import torch
x = torch.empty(5, 3)
print(x)

tensor([[-6.3416e+29,  9.5709e-43,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])


In [3]:
x = torch.rand(5, 3)
print(x)

tensor([[0.4078, 0.0237, 0.1096],
        [0.9834, 0.5038, 0.2657],
        [0.6999, 0.4089, 0.6681],
        [0.1669, 0.5212, 0.1783],
        [0.8387, 0.7319, 0.4652]])


In [4]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [6]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [7]:
# 返回具有相同的torch.dtype和torch.device
x = x.new_ones(5, 3, dtype=torch.float64)
print(x)

# 指定新的数据类型
x = torch.randn_like(x, dtype=torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.6159,  0.6796,  1.1086],
        [ 0.0937, -0.1058,  1.1225],
        [ 0.1414, -0.4780,  0.2334],
        [-1.0955,  1.0423,  1.5510],
        [-0.6971, -2.0237, -0.3580]])


In [8]:
# 获取Tensor形状
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


Tensor很多属性如下：

函数	功能
Tensor(*sizes)	基础构造函数
tensor(data,)	类似np.array的构造函数
ones(*sizes)	全1Tensor
zeros(*sizes)	全0Tensor
eye(*sizes)	对角线为1，其他为0
arange(s,e,step)	从s到e，步长为step
linspace(s,e,steps)	从s到e，均匀切分成steps份
rand/randn(*sizes)	均匀/标准分布
normal(mean,std)/uniform(from,to)	正态分布/均匀分布
randperm(m)	随机排列

# 2.操作

In [9]:
# 1.加法
y = torch.rand(5,3)
print(x + y)

print(torch.add(x, y))

# 可以指定输出
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

# inplace版本后面有_符号
y.add_(x)
print(y)

tensor([[ 0.6171,  0.8436,  1.2384],
        [ 0.7266,  0.2240,  1.8171],
        [ 0.3530,  0.2410,  1.0820],
        [-0.5532,  1.0829,  1.8235],
        [-0.6541, -1.0947,  0.4992]])
tensor([[ 0.6171,  0.8436,  1.2384],
        [ 0.7266,  0.2240,  1.8171],
        [ 0.3530,  0.2410,  1.0820],
        [-0.5532,  1.0829,  1.8235],
        [-0.6541, -1.0947,  0.4992]])
tensor([[ 0.6171,  0.8436,  1.2384],
        [ 0.7266,  0.2240,  1.8171],
        [ 0.3530,  0.2410,  1.0820],
        [-0.5532,  1.0829,  1.8235],
        [-0.6541, -1.0947,  0.4992]])
tensor([[ 0.6171,  0.8436,  1.2384],
        [ 0.7266,  0.2240,  1.8171],
        [ 0.3530,  0.2410,  1.0820],
        [-0.5532,  1.0829,  1.8235],
        [-0.6541, -1.0947,  0.4992]])


In [11]:
# 2.索引
# 注意索引与原来共享内存，一个变化，原来也会跟着变化
y = x[0, :]
y += 1
print(y)
print(x[0, :])  #源Tensor也改变了

tensor([1.6159, 1.6796, 2.1086])
tensor([1.6159, 1.6796, 2.1086])


函数	功能
index_select(input, dim, index)	在指定维度dim上选取，比如选取某些行、某些列
masked_select(input, mask)	例子如上，a[a>0]，使用ByteTensor进行选取
nonzero(input)	非0元素的下标
gather(input, dim, index)	根据index，在dim维度上选取数据，输出的size与index一样

In [12]:
# 3.改变形状
y = x.view(15)
z = x.view(-1, 5)  # -1所指维度可以根据其他维度的值推断出来
print(x.size(), y.size(), z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


# 3. 自动求梯度
PyTorch提供的`autograd包`能够根据输入和前向传播过程自动构建计算图，并执行反向传播。
Tensor如果将其属性`.requires_grad`设置为True，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用`.backward()`来完成所有梯度计算。此Tensor的梯度将累积到`.grad`属性中。

> 注意在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor。

如果不想要被继续追踪，可以调用`.detach()`将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用`with torch.no_grad()`将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。

Function是另外一个很重要的类。Tensor和Function互相结合就可以构建一个记录有整个计算过程的`有向无环图（DAG）`。每个Tensor都有一个`.grad_fn`属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

In [13]:
x = torch.ones(2, 2, requires_grad = True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [14]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [15]:
print(x.is_leaf, y.is_leaf)

True False


In [16]:
z = y * y + 3
out = z.mean()
print(z, out)

tensor([[12., 12.],
        [12., 12.]], grad_fn=<AddBackward0>) tensor(12., grad_fn=<MeanBackward0>)


In [19]:
# in-place方式改变requires_grad属性
a = torch.randn(2, 2) # 默认requires_grad属性为False
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## 3.1梯度
因为out是一个标量，所以调用backward()时不需要指定求导变量.

In [20]:
out.backward() #等价于out.backward(torch.tensor(1.))

In [21]:
print(x.grad) # out关于x的梯度

tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])


In [22]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[2.5000, 2.5000],
        [2.5000, 2.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [23]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad = True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


In [24]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype = torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [25]:
# 梯度追踪的例子
x = torch.tensor(1.0, requires_grad = True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [26]:
# 只改变数值，不会被记录到计算图中，所以不影响梯度
x = torch.ones(1, requires_grad = True)
print(x.data) # 还是一个Tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外
y = 2 * x
x.data *= 100 # 只是改变值，不影响梯度
y.backward()
print(x)
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
